In [2]:

    import numpy as np
    import xarray as xr

    from xlearn22.cluster import KMeans as KMeans22
    from sklearn.metrics import silhouette_score
    from kneed import KneeLocator
    from gap_statistic import OptimalK
    from sklearn.datasets import make_blobs


C:\Users\HP\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\HP\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\HP\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


cek


In [3]:
    n_clusters=15
    annual_cycle=False
    with_pca=False
    

In [6]:
        
        #variasi dan set jumlah kluster pada data sintetik 
        for c in [12]:
            # Generate synthetic data n centers
            X, y = make_blobs(n_samples=7500, n_features=12, centers=c, random_state=42)
            
            # mau rubah n_samples? rubah juga lats, lons dan harus n_samples= lats x lons
           
            lon_values = np.linspace(90, 144, 100)
            lat_values = np.linspace(-10, 7, 75)

            # Create coordinate arrays
            lon, lat = np.meshgrid(lon_values, lat_values)

            # Reshape data to fit the lon/lat grid
            X_reshaped = X.reshape((len(lon_values), len(lat_values), -1))

            # Create xarray DataArray
            data_array = xr.DataArray(X_reshaped, coords=[('lon', lon_values), ('lat', lat_values), ('feature', np.arange(12))])

            dm=data_array.stack(z=("lat", "lon")) 
            
            metrik=[]       
            sse = []
            silhouette_coef = []
           
            for k in range(1, n_clusters):
                print('k=',k)
                kmeans, features = KMeans22(n_clusters=k,  random_state=42).fit3(dm, annual_cycle, with_pca=False)
                                    
                sse.append(kmeans.inertia_) 
               
                if k>1:
                    label2=kmeans.cluster_centers_da.sel(cluster=0).stack(z=("lat", "lon"))
                    label2=label2.dropna(dim=('z'))
                    #print('label2', label2)
                    score = silhouette_score(features, label2)
                else:
                    score=0
                silhouette_coef.append(score)
            
            kl = KneeLocator(
                range(1, n_clusters), sse, curve="convex", direction="decreasing"
                )
            c=kl.elbow
            print('Optimum cluster SSE=',c)
            metrik.append(c)
       
            s=np.array(silhouette_coef)
            c2=np.where(s==s.max())[0][0]+1
            
            print('Optimum cluster SHS=',c2)
            metrik.append(c2)
            X=dm.T
            # Membuang np.NaN (for land only)
            valid_features_index = ~np.isnan(X[:,0])
            X_valid = X[valid_features_index.data,:]
            X=X_valid
            
            from sklearn.cluster import KMeans
            from sklearn.metrics import pairwise_distances_argmin_min
            from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
            
            #CHI
            #--- calinski_harabasz_score
            def find_optimal_clusters(X, max_clusters=n_clusters):
                #Find the optimal number of clusters that maximizes the Calinski-Harabasz index.
                calinski_harabasz_scores = []
                for k in range(2, max_clusters + 1):
                    kmeans = KMeans(n_clusters=k, random_state=42)
                    labels = kmeans.fit_predict(X)
                    ch_score = calinski_harabasz_score(X, labels)
                    calinski_harabasz_scores.append(ch_score)

                # Find the number of clusters with the maximum Calinski-Harabasz index
                optimal_clusters = np.argmax(calinski_harabasz_scores) + 2  # Add 2 because we started from k=2
                return optimal_clusters
                
            # Call
            optimal_clusters = find_optimal_clusters(X)
            print(f"Optimal number of clusters 2: {optimal_clusters}")
            metrik.append(optimal_clusters)
            
            #DBI
            # ini dari chatGPT hasil tidak = yg dari sklearn
            def davies_bouldin_index(X, labels):
                
                #Calculate the Davies-Bouldin index for a given clustering.

                k = len(np.unique(labels))
                #print('len(np.unique(labels))=', k)
                cluster_centers = [np.mean(X[labels == i], axis=0) for i in range(k)]

                # Calculate pairwise distances between cluster centers
                center_distances = pairwise_distances_argmin_min(cluster_centers, cluster_centers)

                # Calculate cluster-wise Davies-Bouldin index
                db_index = 0.0
                for i in range(k):
                    max_ratio = 0.1
                    for j in range(k):
                        if i != j:
                            ratio = (np.sum(pairwise_distances_argmin_min(X[labels == i], X[labels == j])[1]) +
                                     np.sum(pairwise_distances_argmin_min(X[labels == j], X[labels == i])[1])) \
                                     / center_distances[1][i]
                            if ratio > max_ratio:
                                max_ratio = ratio
                    db_index += max_ratio

                return db_index / k

            def find_optimal_clusters(X, max_clusters=n_clusters):
              
                #Find the optimal number of clusters that minimizes the Davies-Bouldin index.
                
                davies_bouldin_scores = []

                for k in range(2, max_clusters + 1):
                    print(k)
                    kmeans = KMeans(n_clusters=k, random_state=42)
                    labels = kmeans.fit_predict(X)
                    #db_score = davies_bouldin_index(X, labels)
                    #ini dari sklearn 
                    db_score = davies_bouldin_score(X, labels)
                    davies_bouldin_scores.append(db_score)

                # Find the number of clusters with the minimum Davies-Bouldin index
                optimal_clusters = np.argmin(davies_bouldin_scores) + 2  
                # Add 2 because we started from k=2
                return optimal_clusters

            # Call
            optimal_clusters = find_optimal_clusters(X)
            print(f"Optimal number of clusters 1: {optimal_clusters}")
            metrik.append(optimal_clusters)
            
            #GP

            optimal_k = OptimalK(parallel_backend='rust')
            #optimal_k = OptimalK(parallel_backend='joblib') #ini hasilnya aneh
            #optimal_k = OptimalK() #ini hasilnya aneh
            optimal_k_clusters = optimal_k(X_valid, cluster_array=np.arange(1, n_clusters), 
                                           n_refs=50)
            
            self=optimal_k
            
            '''
            # Print the optimal number of clusters
            print("Optimal number of clusters:", optimal_k_clusters)

            # Plot Gap Statistics
            #optimal_k.plot_results()

            # Gap values plot
            
            ax[0,i].plot(self.gap_df.n_clusters, self.gap_df.gap_value, linewidth=3)
            ax[0,i].scatter(
            self.gap_df[self.gap_df.n_clusters == self.n_clusters].n_clusters,
            self.gap_df[self.gap_df.n_clusters == self.n_clusters].gap_value,
            s=250,
            c="r",
            )
            
            
            # Gap* plot
            max_ix = self.gap_df[self.gap_df["gap*"] == self.gap_df["gap*"].max()].index[0]
            ax[0,i+1].plot(self.gap_df.n_clusters, self.gap_df["gap*"], linewidth=3)
            ax[0,i+1].scatter(
                self.gap_df.loc[max_ix]["n_clusters"],
                self.gap_df.loc[max_ix]["gap*"],
                s=250,
                c="r",
            )
            plt.show()
            '''
            max_ix = self.gap_df[self.gap_df["gap*"] == self.gap_df["gap*"].max()].index[0]
            
            print("Optimal number of clusters:", self.gap_df.loc[max_ix]["n_clusters"])
            
            metrik.append(self.gap_df.loc[max_ix]["n_clusters"])
            
    print('metrik=', metrik)
            
            
        
   


k= 1
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 2
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 3
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 4
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 5
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 6
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 7
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 8
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 9
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 10
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 11
-------- clustering... fit3 .... ------------------
fit dari sklearn cluster KMeans
k= 12
-------- clus